In [1]:
import pandas as pd
import nltk
import string
import re
import tensorflow as tf
import numpy as np

data =pd.read_csv('data.csv')


In [2]:
data =data.sample(frac=1).reset_index(drop=True)

In [3]:
data.head()

,sentiment,message
0,Bullish,$FB FB and Cos have shown us that we will do A...
1,Bullish,$FB 200 after the Call
2,Bullish,$FB would like to get into this if it goes to ...
3,Bearish,$FB surprised it’s holding 185... thinking 182
4,Bullish,$FB buy here


In [4]:
custom_stop_words = ['i','me', 'my','myself','we','our','ours','ourselves','you',"you're","you've","you'll","you'd",'your','yours','yourself','yourselves','he','him','his','himself','she',"she's",'her','hers','herself','it',"it's",'its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that',"that'll",'these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','then','once','here','there','when','where','why','how','all','any','both','no','nor','not','only','own','same','so','s','t','can','will','just','don',"don't",'should',"should've",'now','d','ll','m','o','re','ve','y','ain','aren',"aren't",'couldn',"couldn't",'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",'ma','mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn',"wouldn't","to"]
stop_words = set(custom_stop_words)

def tokenizer(s):
    s = re.sub(r'http\S+', '', s)
    s = re.sub(r'\$(\w+)','',s)
    translate_table = dict((ord(char), None) for char in string.punctuation)   
    s = s.translate(translate_table)
    tokens = nltk.word_tokenize(s)
    filtered_tokens =[]
    for word in tokens:
        if word.lower() not in stop_words:
            filtered_tokens.append(word.lower())
    return filtered_tokens

data['message'] = data['message'].apply(tokenizer)
data['message_length'] = data['message'].apply(len)
data = data[(data['message_length']>0)]

def func(x):
    if x=='Bullish':
        return 1
    return 0

data.sentiment = data.sentiment.apply(func)
data.head()

,sentiment,message,message_length
0,1,"[fb, cos, shown, us, anything, recognition, fa...",12
1,1,"[200, after, call]",3
2,1,"[would, like, get, into, goes]",5
3,0,"[surprised, ’, holding, 185, thinking, 182]",6
4,1,[buy],1


In [5]:
data.message_length.value_counts()

3     14796
4     14113
5     13324
2     13072
6     12273
7     11271
8     10475
9      9834
10     8972
11     8440
12     8010
13     7677
14     7325
1      7248
15     6258
16     4930
17     3629
18     2443
19     1393
20      690
21      322
22      136
23       52
24       24
26        9
25        8
27        2
29        1
28        1
44        1
Name: message_length, dtype: int64

In [6]:
words= []
for message in data.message:
    for word in message:
        words.append(word)

from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

In [7]:
len(vocab_to_int)

70116

In [9]:
max_len = 25


def fun(x):
    vec = []
    zeroes = max_len - len(x)
    if zeroes>0:
        for i in range(zeroes):
            vec.append(0)
    
    for i in range(max_len-len(vec)):
        word = x[i]
        vec.append(vocab_to_int[word])
    return np.array(vec)

data['encoded_message'] = data['message'].apply(fun)
data.head()

,sentiment,message,message_length,encoded_message
0,1,"[fb, cos, shown, us, anything, recognition, fa...",12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24..."
1,1,"[200, after, call]",3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,"[would, like, get, into, goes]",5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0,"[surprised, ’, holding, 185, thinking, 182]",6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,[buy],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
features= []
for vec in data.encoded_message:
    features.append(vec.tolist())
features =np.array(features)
features = features[:150000]
print(features)
features.shape

[[    0     0     0 ...,    11   598 25780]
 [    0     0     0 ...,   179    31   109]
 [    0     0     0 ...,    14    52   175]
 ..., 
 [    0     0     0 ...,    50    66   420]
 [    0     0     0 ...,   192   357  1342]
 [    0     0     0 ...,    25    17   374]]


(150000, 25)

In [15]:
split_frac = 0.7

split_index = int(split_frac * len(features))

train_x, val_x = features[:split_index], features[split_index:] 
train_y, val_y = data.sentiment[:split_index], data.sentiment[split_index:len(features)]

split_frac = 0.2
split_index = int(split_frac * len(val_x))

val_x, test_x = val_x[:split_index], val_x[split_index:]
val_y, test_y = val_y[:split_index], val_y[split_index:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(105000, 25) 
Validation set: 	(9000, 25) 
Test set: 		(36000, 25)
label set: 		(105000,) 
Validation label set: 	(9000,) 
Test label set: 		(36000,)


In [16]:
lstm_size = 256
lstm_layers = 2
batch_size = 1000
learning_rate = 0.01

n_words = len(vocab_to_int) + 1 # Add 1 for 0 added to vocab

# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, None], name="inputs")
    labels_ = tf.placeholder(tf.int32, [None, None], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    
embed_size = 300 

with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)
    
def lstm_cell():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    # Add dropout to the cell
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

with tf.name_scope("RNN_layers"):
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    tf.summary.histogram('predictions', predictions)
with tf.name_scope('cost'):
    cost = tf.losses.mean_squared_error(labels_, predictions)
    tf.summary.scalar('cost', cost)

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

merged = tf.summary.merge_all()

with tf.name_scope('validation'):
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]


In [17]:
saver = tf.train.Saver()

In [13]:
epochs = 30

# with graph.as_default():
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/tb/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/tb/test', sess.graph)
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            summary, loss, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)
#             loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)

            train_writer.add_summary(summary, iteration)
        
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
#                     batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
            test_writer.add_summary(summary, iteration)
            saver.save(sess, "checkpoints/sentiment_jatin.ckpt")
    saver.save(sess, "checkpoints/sentiment_jatin.ckpt")

Epoch: 0/30 Iteration: 5 Train loss: 0.278
Epoch: 0/30 Iteration: 10 Train loss: 0.227
Epoch: 0/30 Iteration: 15 Train loss: 0.216
Epoch: 0/30 Iteration: 20 Train loss: 0.214
Epoch: 0/30 Iteration: 25 Train loss: 0.213
Val acc: 0.682
Epoch: 0/30 Iteration: 30 Train loss: 0.205
Epoch: 0/30 Iteration: 35 Train loss: 0.197
Epoch: 0/30 Iteration: 40 Train loss: 0.187
Epoch: 0/30 Iteration: 45 Train loss: 0.177
Epoch: 0/30 Iteration: 50 Train loss: 0.186
Val acc: 0.764
Epoch: 0/30 Iteration: 55 Train loss: 0.170
Epoch: 0/30 Iteration: 60 Train loss: 0.182
Epoch: 0/30 Iteration: 65 Train loss: 0.162
Epoch: 0/30 Iteration: 70 Train loss: 0.159
Epoch: 0/30 Iteration: 75 Train loss: 0.158
Val acc: 0.778
Epoch: 1/30 Iteration: 80 Train loss: 0.148
Epoch: 1/30 Iteration: 85 Train loss: 0.158
Epoch: 1/30 Iteration: 90 Train loss: 0.146
Epoch: 1/30 Iteration: 95 Train loss: 0.145
Epoch: 1/30 Iteration: 100 Train loss: 0.146
Val acc: 0.793
Epoch: 1/30 Iteration: 105 Train loss: 0.124
Epoch: 1/30 Ite

Epoch: 11/30 Iteration: 860 Train loss: 0.037
Epoch: 11/30 Iteration: 865 Train loss: 0.030
Epoch: 11/30 Iteration: 870 Train loss: 0.038
Epoch: 11/30 Iteration: 875 Train loss: 0.027
Val acc: 0.780
Epoch: 11/30 Iteration: 880 Train loss: 0.033
Epoch: 11/30 Iteration: 885 Train loss: 0.029
Epoch: 11/30 Iteration: 890 Train loss: 0.032
Epoch: 11/30 Iteration: 895 Train loss: 0.029
Epoch: 11/30 Iteration: 900 Train loss: 0.039
Val acc: 0.784
Epoch: 11/30 Iteration: 905 Train loss: 0.032
Epoch: 11/30 Iteration: 910 Train loss: 0.031
Epoch: 11/30 Iteration: 915 Train loss: 0.035
Epoch: 11/30 Iteration: 920 Train loss: 0.031
Epoch: 12/30 Iteration: 925 Train loss: 0.027
Val acc: 0.785
Epoch: 12/30 Iteration: 930 Train loss: 0.032
Epoch: 12/30 Iteration: 935 Train loss: 0.023
Epoch: 12/30 Iteration: 940 Train loss: 0.034
Epoch: 12/30 Iteration: 945 Train loss: 0.037
Epoch: 12/30 Iteration: 950 Train loss: 0.028
Val acc: 0.785
Epoch: 12/30 Iteration: 955 Train loss: 0.024
Epoch: 12/30 Iterati

Epoch: 21/30 Iteration: 1685 Train loss: 0.029
Epoch: 21/30 Iteration: 1690 Train loss: 0.024
Epoch: 22/30 Iteration: 1695 Train loss: 0.023
Epoch: 22/30 Iteration: 1700 Train loss: 0.035
Val acc: 0.779
Epoch: 22/30 Iteration: 1705 Train loss: 0.028
Epoch: 22/30 Iteration: 1710 Train loss: 0.036
Epoch: 22/30 Iteration: 1715 Train loss: 0.034
Epoch: 22/30 Iteration: 1720 Train loss: 0.024
Epoch: 22/30 Iteration: 1725 Train loss: 0.027
Val acc: 0.779
Epoch: 22/30 Iteration: 1730 Train loss: 0.028
Epoch: 22/30 Iteration: 1735 Train loss: 0.027
Epoch: 22/30 Iteration: 1740 Train loss: 0.023
Epoch: 22/30 Iteration: 1745 Train loss: 0.029
Epoch: 22/30 Iteration: 1750 Train loss: 0.032
Val acc: 0.781
Epoch: 22/30 Iteration: 1755 Train loss: 0.020
Epoch: 22/30 Iteration: 1760 Train loss: 0.028
Epoch: 22/30 Iteration: 1765 Train loss: 0.030
Epoch: 22/30 Iteration: 1770 Train loss: 0.037
Epoch: 23/30 Iteration: 1775 Train loss: 0.030
Val acc: 0.781
Epoch: 23/30 Iteration: 1780 Train loss: 0.024


In [18]:
test_acc = []
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/sentiment_jatin.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/sentiment_jatin.ckpt
Test accuracy: 0.695
